In [1]:
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.image import imread
from pathlib import Path
from skimage.io import imread
from PIL import Image
from typing import Union, Tuple

#------ALL FUNCTIONS NEEDED------


# Gamma transformation
def gammatransformation(image, gamma=None):
   """
    Every pixel value p is transformed by p^gamma.

    Parameters
    ----------
    parameter1 : gray level image
        A 2D array containing all the gray levels of each pixel.

    Returns 
    -------
    new image as 2D array.
       Each pixel value of the transformed image is result of p^gamma.
       Result is in the range 0-1
    """
   
   
   # Check if there are negative values in the array and if so, shift into positive range
   if np.any(image<0):
       image = image + abs(np.min(image))

   # Scale to 0-1
   if image.min() != 0 or image.max() != 1:
    image = (image - image.min()) / (image.max() - image.min())
    
   # Define gamma based on mean illumination
   if gamma is None:
     if np.mean(image) > 0.5:
        gamma = 1.5
     else:
       gamma = 0.1


   # Perform gamma transformation
   img_gamma = np.power(image, gamma)

   # Scale back to 0-255 8 bit
   img_gamma = (img_gamma * 255).astype(np.uint8)

   return img_gamma

# creating histogram for Otsu threshholding  
def custom_histogram(image: np.ndarray, nbins: int = 256) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computes the histogram and corresponding bin centers of a grayscale image,
    replicating the behavior of skimage.exposure.histogram, including normalization
    to the [0, 255] range. This ensures consistent behavior with Otsu implementations
    that assume 8-bit images.

    Args:
        image (np.ndarray): Input image as a 2D array of grayscale values.
        nbins (int): Number of bins for the histogram (default: 256).

    Returns:
        hist (np.ndarray): Array of histogram frequencies for each bin.
        bin_centers (np.ndarray): Array of bin center values.
    """
    # Determine the minimum and maximum pixel intensity in the image
    img_min, img_max = image.min(), image.max()

    # Normalize the image intensities to the range [0, 255], as in skimage
    image_scaled = ((image - img_min) / (img_max - img_min) * 255).astype(np.uint8)

    # Compute the histogram of the scaled image within [0, 255]
    hist, bin_edges = np.histogram(
        image_scaled.ravel(),
        bins=nbins,
        range=(0, 255)
    )

    # Compute bin centers as the average of adjacent bin edges
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    return hist, bin_centers

# Otsu thresholding
def apply_global_otsu(image: np.ndarray) -> float:
    """
    Computes the global Otsu threshold of an input grayscale image in a way that matches
    the behavior of skimage.filters.threshold_otsu, including histogram scaling and
    threshold rescaling back to the original intensity range.

    This function enables nearly identical thresholding results to skimage's implementation,
    even on images with floating-point or non-8-bit integer data.

    Args:
        image (np.ndarray): Input image as a 2D array of grayscale values.

    Returns:
        threshold_original (float): Computed Otsu threshold mapped back to the original image range.
    """
    # Compute histogram and bin centers consistent with skimage
    hist, bin_centers = custom_histogram(image, nbins=256)
    hist = hist.astype(np.float64)

    # Normalize histogram to obtain probability distribution p(k)
    p = hist / hist.sum()

    # Compute cumulative sums of class probabilities ω0 and ω1
    omega0 = np.cumsum(p)                           # Class probabilities for background
    omega1 = np.cumsum(p[::-1])[::-1]               # Class probabilities for foreground

    # Compute cumulative sums of class means μ0 and μ1
    mu0 = np.cumsum(p * bin_centers)                # Class means for background
    mu1 = np.cumsum((p * bin_centers)[::-1])[::-1]  # Class means for foreground

    # Compute between-class variance σ_b^2 for each possible threshold
    sigma_b_squared = (omega0[:-1] * omega1[1:] * (mu0[:-1] / omega0[:-1] - mu1[1:] / omega1[1:])**2)

    # Find the threshold index t maximizing σ_b^2
    t_idx = np.argmax(sigma_b_squared)
    t_scaled = bin_centers[t_idx]

    # Rescale threshold t back to original image intensity range
    img_min, img_max = image.min(), image.max()
    t_original = t_scaled / 255 * (img_max - img_min) + img_min

    return (image > t_original).astype(np.uint8)

# Dice score
def dice_score(otsu_img, otsu_gt):

    # control if the Pictures have the same Size
    if len(otsu_img) != len(otsu_gt):
       if len(otsu_img) > len(otsu_gt):
         otsu_img = otsu_img[1:len(otsu_gt)]
       else:
        otsu_gt = otsu_gt[1:len(otsu_img)]


    # defining the variables for the Dice Score equation
    positive_overlap = 0
    sum_img = 0
    sum_gt = 0

    for t, p in zip(otsu_img, otsu_gt):
        if t == 1:
            sum_img += 1
        if p == 1:
            sum_gt += 1
        if t == 1 and p == 1:
            positive_overlap += 1

    if sum_img + sum_gt == 0:
        return 1.0

    return 2 * positive_overlap / (sum_img + sum_gt)

# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, g=None) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    img_scaled  = (img / img.max() * 255).astype('uint8')
    gt = imread(gt_path, as_gray=True)
    gt  = 1 - (gt == 0)         

    # Gamma transformation
    img_gamma = gammatransformation(img, gamma=g)
    
    

    # 1nd Otsu
    binary1 = apply_global_otsu(img_gamma)

    # 5. Compute Dice score
    return dice_score(binary1.flatten(), gt.flatten())

# -------------------------------------------------------------------
# Mainroutine: Going through all image-gt pairs and collect all dice scores of data set: NIH3T3
# -------------------------------------------------------------------

# Define gamma values to test (e.g., from 0.1 to 2.0 in steps of 0.1)
gamma_values = np.arange(-2.0, 2.1, 0.1)

# Set paths
img_dir = Path("data/NIH3T3/img")
gt_dir  = Path("data/NIH3T3/gt")

# Loop over gamma values
dice_score_means_NIH3T3 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("dna-*.png")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"{idx}.png"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_NIH3T3.append(dice_score_mean)

np.save('Dice_scores/NIH3T3_dice_means_gamma', dice_score_means_NIH3T3)
    
# NEXT

# Set paths
img_dir = Path("data/N2DL-HeLa/img")
gt_dir  = Path("data/N2DL-HeLa/gt")

# Loop over gamma values
dice_score_means_N2DL_HeLa = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DL_HeLa.append(dice_score_mean)

np.save('Dice_scores/N2DL-HeLa_dice_means_gamma', dice_score_means_N2DL_HeLa)


# NEXT

# Set paths
img_dir = Path("data/N2DH-GOWT1/img")
gt_dir  = Path("data/N2DH-GOWT1/gt")

# Loop over gamma values
dice_score_means_N2DH_GOWT1 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DH_GOWT1.append(dice_score_mean)

np.save('Dice_scores/N2DH-GOWT1_dice_means_gamma', dice_score_means_N2DH_GOWT1)

/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered i

Gamma -2.00 → Mean Dice: 0.2915


/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered i

Gamma -1.90 → Mean Dice: 0.3063


/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered i

...

Gamma -1.60 → Mean Dice: 0.2844


/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered i

Gamma -1.50 → Mean Dice: 0.2737


/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered in cast
  img_gamma = (img_gamma * 255).astype(np.uint8)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:47: RuntimeWarning: divide by zero encountered in power
  img_gamma = np.power(image, gamma)
/var/folders/st/cp4zctjj60n1b98kgyxcm1r00000gn/T/ipykernel_36398/216437128.py:50: RuntimeWarning: invalid value encountered i

KeyboardInterrupt: 

In [18]:
test = imread('data/NIH3T3/img/dna-0.png', as_gray=True)
gt = imread('data/NIH3T3/gt/0.png', as_gray=True)
gt = 1 - (gt==0)

test_gamma = gammatransformation(test,)
print(np.mean(test_gamma))

binary = apply_global_otsu(test)

print(np.mean(binary))

dice = process_single('data/NIH3T3/img/dna-0.png', 'data/NIH3T3/gt/0.png')
print(dice)

191.74891444614954
0.15291486467633927
0.96362949886221
